# Prompting Principles

## Principle 1: Write clear and specific instructions

### Tactics

* Tactic 1: Use delimiters to clearly indicate distinct parts of the input
* Tactic 2: Ask for a structured output
* Tactic 3: Ask the model to check whether conditions are satisfied
* Tactic 4: "Few-shot" prompting

In [1]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message["content"]

## Principle 2: Give the model time to “think” 

### Tactic 1: Specify the steps required to complete a task

In [4]:
text = f"""
In a quiet village nestled between rolling hills, a lone scarecrow stood watch over the golden fields. \
Every day, it gazed at the passing clouds, dreaming of adventures beyond the horizon. One windy night, \
a mischievous crow named Oliver landed on the scarecrow's outstretched arm, seeking shelter from the storm. \
Surprised but curious, the scarecrow listened intently as Oliver recounted tales of faraway lands and daring escapades. \
Inspired by the crow's stories, the scarecrow yearned to experience the world beyond its post. \
With a newfound sense of purpose, the scarecrow bid farewell to the fields and set off into the unknown, \
guided by the stars and fueled by dreams of grand adventures. And so, the unlikely duo of scarecrow and crow \
embarked on a journey that would forever change their destinies, teaching them the true meaning of friendship, \
courage, and the endless possibilities of a brave heart.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Urdu.
3 - List each name in the Urdu summary.
4 - Output a json object that contains the following \
keys: urdu_summary, num_names.

Separate your answers with line breaks.

Text:```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - A lone scarecrow in a quiet village meets a mischievous crow named Oliver and together they embark on a journey of friendship and courage.
2 - ایک آکیلا بھٹیے میدان میں ایک شرارتی کوا جو اولیور کہلاتا ہے سے ملتا ہے اور مل کر دوستی اور بہادری کی سفر پر نکلتے ہیں۔
3 - بھٹیا، اولیور
4 - 
{
  "urdu_summary": "ایک آکیلا بھٹیے میدان میں ایک شرارتی کوا جو اولیور کہلاتا ہے سے ملتا ہے اور مل کر دوستی اور بہادری کی سفر پر نکلتے ہیں۔",
  "num_names": 2
}


#### Ask for output in a specified format

In [5]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into Urdu.
3 - List each name in the Urdu summary.
4 - Output a json object that contains the 
  following keys: f_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: A scarecrow and a crow form an unlikely friendship and set off on a journey together, discovering the true meaning of friendship and courage. 
Translation: ایک خوفناک اور ایک کوا ایک غیر متوقع دوستی قائم کرتے ہیں اور ایک ساتھ سفر پر نکل جاتے ہیں، دوستی اور حوصلہ کا حقیقی مطلب دیکھ کر. 
Names: scarecrow (خوفناک), crow (کوا)
Output JSON: {"f_summary": "A scarecrow and a crow form an unlikely friendship and set off on a journey together, discovering the true meaning of friendship and courage.", "num_names": 2}


### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [6]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is not entirely correct. The total cost calculation should not include the land cost because it is a one-time cost and not part of the yearly operations cost. The correct total cost should be:

Total cost: 250x + 100,000 + 100x = 350x + 100,000

So the correct total cost for the first year of operations as a function of the number of square feet is 350x + 100,000.


In [8]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = ($100 * x) + ($250 * x) + ($100,000 + ($10 * x))
Total cost = $100x + $250x + $100,000 + $10x 
Total cost = $360x + $100,000

My solution:
Total cost = $360x + $100,000
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
incorrect
